In [1]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=560b39f25a6aea716680635a4d2b9cd2b22d10db0c81ed64dac1f1b8bf3acbe5
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [2]:
import os
import sys
import numpy as np
import tarfile
import wget
import warnings
warnings.filterwarnings("ignore")
import tensorflow
import keras

from zipfile import ZipFile
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.initializers import Constant

print("tensorflow", tensorflow.__version__)
print("keras", keras.__version__)

tensorflow 2.12.0
keras 2.12.0


## Store external datasets in local environments

In [3]:
%%capture
try:

    from google.colab import files

    !wget -P DATAPATH http://nlp.stanford.edu/data/glove.6B.zip
    !unzip DATAPATH/glove.6B.zip -d DATAPATH/glove.6B

    !wget -P DATAPATH http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
    !tar -xvf DATAPATH/aclImdb_v1.tar.gz -C DATAPATH

    BASE_DIR = 'DATAPATH'

except ModuleNotFoundError:

    if not os.path.exists('Data/glove.6B'):
        os.mkdir('Data/glove.6B')

        url='http://nlp.stanford.edu/data/glove.6B.zip'
        wget.download(url,'Data')

        temp='Data/glove.6B.zip'
        file = ZipFile(temp)
        file.extractall('Data/glove.6B')
        file.close()



    if not os.path.exists('Data/aclImdb'):

        url='http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
        wget.download(url,'Data')

        temp='Data/aclImdb_v1.tar.gz'
        tar = tarfile.open(temp, "r:gz")
        tar.extractall('Data')
        tar.close()

    BASE_DIR = 'Data'

In [4]:
GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')
TRAIN_DATA_DIR = os.path.join(BASE_DIR, 'aclImdb/train')
TEST_DATA_DIR = os.path.join(BASE_DIR, 'aclImdb/test')

In [5]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

## Load and preprocess data

In [6]:
# Load data from dataset stored in notebook
def get_data(data_dir):
    texts = []  # list of text samples
    labels_index = {'pos':1, 'neg':0}  # dictionary mapping label name to numeric id
    labels = []  # list of label ids
    for name in sorted(os.listdir(data_dir)):
        path = os.path.join(data_dir, name)
        if os.path.isdir(path):
            if name=='pos' or name=='neg':
                label_id = labels_index[name]
                for fname in sorted(os.listdir(path)):
                        fpath = os.path.join(path, fname)
                        text = open(fpath,encoding='utf8').read()
                        texts.append(text)
                        labels.append(label_id)
    return texts, labels

train_texts, train_labels = get_data(TRAIN_DATA_DIR)
test_texts, test_labels = get_data(TEST_DATA_DIR)
labels_index = {'pos':1, 'neg':0}

#Just to see how the data looks like.
print(train_texts[0])
print(train_labels[0])
print(test_texts[24999])
print(test_labels[24999])

Story of a man who has unnatural feelings for a pig. Starts out with a opening scene that is a terrific example of absurd comedy. A formal orchestra audience is turned into an insane, violent mob by the crazy chantings of it's singers. Unfortunately it stays absurd the WHOLE time with no general narrative eventually making it just too off putting. Even those from the era should be turned off. The cryptic dialogue would make Shakespeare seem easy to a third grader. On a technical level it's better than you might think with some good cinematography by future great Vilmos Zsigmond. Future stars Sally Kirkland and Frederic Forrest can be seen briefly.
0
I've seen this story before but my kids haven't. Boy with troubled past joins military, faces his past, falls in love and becomes a man. The mentor this time is played perfectly by Kevin Costner; An ordinary man with common everyday problems who lives an extraordinary conviction, to save lives. After losing his team he takes a teaching posi

In [7]:
# Vectorize the texts training samples into a 2D array using keras Tokenizer
# First keras Tokenizer is fit with training data, and then with tokenize both, train and test data
  # What fit means in this context is to build an internal vocabulary based on the words or tokens present in the training data
  # it also determines the tokenization rules as how to split text into individual words and subwords.
  # Once the rules and the internal vocabulary are set, the tokenizer can be used to tokenize train and test data, which means to break down
  # the text into individual tokens and map them with their own numerical representations, so we can use them as inputs to feet ML models

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_texts)
train_sequences = tokenizer.texts_to_sequences(train_texts) # Converting texts to vectors of word indexes
test_sequences = tokenizer.texts_to_sequences(test_texts)
word_index=tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


Found 88582 unique tokens.


In [8]:
# run a process of zero-padding until vector is of size MAX_SEQUENCE_LENGTH
  # now train sequences is a list of sequences where each one is an array of numeric
  # values where each numeric value represents a word from the internal dictionary
  # created during the fitting process.
  # (i.e)[[["hello world"]], [["I am learning NLP"]]] = [[[23, 12]],[[122, 34, 63, 754]]]
  # this pad_sequences allows to create standard sequences of same length to feed the neural network

train_valid_data = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
train_valid_labels = to_categorical(np.asarray(train_labels))
test_labels = to_categorical(np.asarray(test_labels))

In [13]:
# split the training data into training set and validation set
indices = np.arange(train_valid_data.shape[0])
np.random.shuffle(indices)

# notice that indices is a 1-dimension array
# we are using it to preserve the 2-dimension array in train_valid_data/labels
# test[1] returns 1-dim view of the row in position 1, test[[1]] returns a new array
# that contains the row in position 1.
train_valid_data = train_valid_data[indices]
train_valid_labels = train_valid_labels[indices]

num_validation_samples = int(VALIDATION_SPLIT * train_valid_data.shape[0])

x_train=train_valid_data[:-num_validation_samples]
y_train=train_valid_labels[:-num_validation_samples]

x_val = train_valid_data[-num_validation_samples:]
y_val = train_valid_labels[-num_validation_samples:]

print("x_train", x_train.shape)
print("y_train", y_train.shape)

print("x_test", x_test.shape)
print("y_test", y_test.shape)

x_train (20000, 1000)
y_train (20000, 2)
x_test (5000, 1000)
y_test (5000, 2)


In [14]:
# Create an index mapping words dictionary with Stanford's GloVe 100d word embeddings
embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'),encoding='utf8') as f:
    for line in f:
        # turns doc line into an array for words (["google 23 123 542 .. 324"])
        values = line.split()
        word = values[0] # takes the word
        coefs = np.asarray(values[1:], dtype='float32') # takes the word embedding
        embeddings_index[word] = coefs

print('Found %s word vectors in Glove embeddings.' % len(embeddings_index))
print("Embedding vector for word 'google'", embeddings_index["google"])

Found 400000 word vectors in Glove embeddings.
Embedding vector for word 'google' [ 0.22575  -0.56253  -0.05156  -0.079389  1.1876   -0.48397  -0.23342
 -0.85278   0.97495  -0.33344   0.71692   0.12644   0.31962  -1.4136
 -0.57903  -0.037286 -0.0164    0.45155  -0.29005   0.52599  -0.22534
 -0.29556  -0.032407  1.5608   -0.013499 -0.064558  0.26625   0.78595
 -0.71693  -0.93025   0.80461   1.6035   -0.30602  -0.34764   0.93872
  0.38137  -0.26743  -0.56519   0.58899  -0.14554  -0.34324   0.21291
 -0.39887   0.090042 -0.8495    0.38803  -0.5045   -0.22488   1.0644
 -0.2624    1.0334    0.06348  -0.39989   0.24236  -0.65636  -1.8107
 -0.061801  0.13795   1.1658   -0.30046  -0.50143   0.16509   0.039835
  0.62541   0.56935   0.64125   0.21308   0.30276   0.39673   0.38973
  0.28183   0.79481  -0.11962  -0.49598  -0.53195  -0.14897   0.51254
 -0.39208  -0.58535  -0.078509  0.81721  -0.73497  -0.68131   0.099243
 -0.87608   0.029632  0.33402  -0.14305   0.16964  -0.035178  0.39777
  0.71769

In [15]:
# prepare embedding matrix - rows are the words from word_index, columns are the embeddings of that word from glove.
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


# load these pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

print("Preparing of embedding matrix is done")

Preparing of embedding matrix is done


## 1D CNN Model with pre-trained embedding

In [16]:
cnnmodel = Sequential()
cnnmodel.add(embedding_layer)
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(GlobalMaxPooling1D())
cnnmodel.add(Dense(128, activation='relu'))
cnnmodel.add(Dense(len(labels_index), activation='softmax'))

cnnmodel.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
#Train the model. Tune to validation set.
cnnmodel.fit(x_train, y_train,
          batch_size=128,
          epochs=1, validation_data=(x_val, y_val))
#Evaluate on test set:
score, acc = cnnmodel.evaluate(test_data, test_labels)
print('Test accuracy with CNN:', acc)

782/782 [==============================] - 4s 5ms/step - loss: 0.5578 - acc: 0.7138
Test accuracy with CNN: 0.7138000130653381


## 1D CNN Model with training your own embeddings

In [17]:
print("Defining and training a CNN model, training embedding layer on the fly instead of using pre-trained embeddings")
cnnmodel = Sequential()
cnnmodel.add(Embedding(MAX_NUM_WORDS, 128))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(MaxPooling1D(5))
cnnmodel.add(Conv1D(128, 5, activation='relu'))
cnnmodel.add(GlobalMaxPooling1D())
cnnmodel.add(Dense(128, activation='relu'))
cnnmodel.add(Dense(len(labels_index), activation='softmax'))

cnnmodel.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
#Train the model. Tune to validation set.
cnnmodel.fit(x_train, y_train,
          batch_size=128,
          epochs=1, validation_data=(x_val, y_val))
#Evaluate on test set:
score, acc = cnnmodel.evaluate(test_data, test_labels)
print('Test accuracy with CNN:', acc)

Defining and training a CNN model, training embedding layer on the fly instead of using pre-trained embeddings
782/782 [==============================] - 4s 6ms/step - loss: 0.7841 - acc: 0.6436
Test accuracy with CNN: 0.6435999870300293


## LSTM Model with training your own embeddings

In [18]:
rnnmodel = Sequential()
rnnmodel.add(Embedding(MAX_NUM_WORDS, 128))
rnnmodel.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
rnnmodel.add(Dense(2, activation='sigmoid'))
rnnmodel.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print('Training the RNN')

rnnmodel.fit(x_train, y_train,
          batch_size=32,
          epochs=1,
          validation_data=(x_val, y_val))
score, acc = rnnmodel.evaluate(test_data, test_labels,
                            batch_size=32)
print('Test accuracy with RNN:', acc)

Training the RNN
782/782 [==============================] - 234s 299ms/step - loss: 0.3833 - accuracy: 0.8384
Test accuracy with RNN: 0.8383600115776062


## LSTM model using pre-trained embedding layer

In [19]:
rnnmodel2 = Sequential()
rnnmodel2.add(embedding_layer)
rnnmodel2.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
rnnmodel2.add(Dense(2, activation='sigmoid'))
rnnmodel2.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print('Training the RNN')

rnnmodel2.fit(x_train, y_train,
          batch_size=32,
          epochs=1,
          validation_data=(x_val, y_val))
score, acc = rnnmodel2.evaluate(test_data, test_labels,
                            batch_size=32)
print('Test accuracy with RNN:', acc)

Training the RNN
782/782 [==============================] - 235s 301ms/step - loss: 0.5082 - accuracy: 0.7601
Test accuracy with RNN: 0.7600799798965454
